In [1]:
import sys
import os

In [2]:
from importer import *
from utils import *

In [3]:
# for plots

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [4]:
CLIPS_ROOT = raw_clip_data
SAVE_ROOT = os.path.join(save_clip_data, 'mask_pose')

In [5]:
from eventmap import *

mappingtool = MappingTool()

Show all events:

In [6]:
events = [event_names[i] for i in event_ids]
events

['meal time',
 'meeting',
 'coffee break',
 'nap',
 'cooking',
 'working',
 'tending to plants']

## Mark

select an event

In [33]:
event = events[6]

Initialize clip management

In [34]:
# file management
manager = ClipsSavedDataManagement(event=event, root_path=SAVE_ROOT)

# all of the clips
clips = manager.change_event(event)

Using Event tending to plants
Available clips
['0', '1', '2', '3', '4', '5', '6', '7']
Using Event tending to plants
Available clips
['0', '1', '2', '3', '4', '5', '6', '7']


for each clip, save an image of the pose and event maps

In [35]:
save_directory = os.path.join(os.getcwd(), 'images', f'{event}')
save_directory

'/home/haruyaishikawa/dev/action_mapping/notebook/images/tending to plants'

In [10]:
os.mkdir(save_directory)

FileExistsError: [Errno 17] File exists: '/home/haruyaishikawa/dev/action_mapping/notebook/images/meal time'

In [36]:
for clip in clips:
    
    all_files = manager.get_sorted_npz_files(clip)
    
    print(f'{clip}: ', len(all_files))

    # initialize averages
    pose_averages = {}
    object_averages = {}
    points = np.empty((3, 1))
    points[:] = np.nan
    H = mappingtool.single_histogram(points)
    for name, value in eventmap_pose_dict.items():
        pose_averages[name] = H.copy()
    for name, value in eventmap_object_dict.items():
        object_averages[name] = H.copy()
   
    
    for file in all_files:
        
        filename = os.path.join(manager.get_clip_directory(clip), file)
        
        # get poses and masks
        poses, masks = poses_masks_from_npz(filename)
        
        # process poses
        if not(poses is None):
            for i, pose in poses.items():
                # for each people
                for name, joints in eventmap_pose_dict.items():
                    pose_points = process_body_part(pose, joints)
                    pose_H = mappingtool.single_histogram(pose_points)
                    
                    pose_averages[name] += pose_H
        
        # process masks
        if not(masks is None):       
            for mask in masks:

                object_id = int(mask.split('_')[0])

                for object_name, object_ids in eventmap_object_dict.items():
                    if object_id in object_ids:
                        mask = masks[mask]

                        mask_points = process_object(mask)
                        mask_H = mappingtool.single_histogram(mask_points)

                        object_averages[object_name] += mask_H
    
    # saving for each clips
    colormap = 'plasma'
    X, Y = np.meshgrid(mappingtool.xedges, mappingtool.yedges)
    
    # figure for pose
    plt.ioff()
    fig, axs = plt.subplots(1, len(eventmap_pose_dict), 
                            figsize=(20, 5), facecolor='w', edgecolor='k')
    # fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()
    axs_index = 0
    
    for name, pose_H in pose_averages.items():
        pose_H = mappingtool.normalize(pose_H)
        axs[axs_index].pcolormesh(X, Y, pose_H, cmap=colormap, vmin=0.0, vmax=1.0)
        axs[axs_index].set_title(name, fontsize=24)
        axs_index += 1
    
    pose_plot = os.path.join(save_directory, f'{clip}_pose.png')
    fig.savefig(pose_plot, dpi=fig.dpi)
    plt.close()
    
    # figure for objects
    rows = 3
    fig, axs = plt.subplots(rows+1, len(eventmap_object_dict)//rows, 
                            figsize=(20, 20), facecolor='w', edgecolor='k')
    # fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()
    axs_index = 0
    
    for name, mask_H in object_averages.items():
        mask_H = mappingtool.normalize(mask_H)
        axs[axs_index].pcolormesh(X, Y, mask_H, cmap=colormap, vmin=0.0, vmax=1.0)
        axs[axs_index].set_title(name, fontsize=24)
        axs_index += 1 
    
    mask_plot = os.path.join(save_directory, f'{clip}_objects.png')
    fig.savefig(mask_plot, dpi=fig.dpi)
    plt.close()

0:  701
1:  747
2:  516
3:  744
4:  756
5:  746
6:  701
7:  590
